In [1]:
from EMS import EMS
from Organization import Organization
from Energy import Energy
from Hierarchy import Hierarchy
from Dataflow import DataFlow
import pandas as pd
import numpy as np

from EMS import EMS
from Organization import Organization
from Energy import Energy
from Hierarchy import Hierarchy
from Dataflow import DataFlow

import xml.etree.ElementTree as ET
from common import readOption, logger
from IPython.display import display
# Set the maximum number of columns to be displayed to None
pd.set_option('display.max_columns', None)

In [2]:
# Specify the path to your CSV file
# filepath = "./output/vtchk/sys_df_after.csv"
# filepath = "./output/vtchk/sys_df.csv"
filepath = "./output/vtchk/result_df.csv"



# Read the CSV file into a dataframe using read_csv()
df = pd.read_csv(filepath)
df[:3]

,Unnamed: 0,id_x,parent_system_id,name_x,source_key,meter_id,composition_expression,component_of_id,company_id,city_id,city_name,id_y,ref_id,name_y,tenant_id,company,company_code
0,0,16115,NaN,VTC,NaN,NaN,NaN,NaN,311,NaN,NaN,NaN,NaN,NaN,7d353579-07af-5479-bd50-14333cf4eb76,VTC,vtchk
1,1,16127,16124.0,Sockets and others,Sockets and other,625.0,NaN,NaN,311,2.0,Hong Kong,cf7986d7-25f1-48b0-90c1-c16f44371c1a,16127.0,Sockets and other,7d353579-07af-5479-bd50-14333cf4eb76,VTC,vtchk
2,2,16148,16120.0,IAQ Sensor 1101,NaN,NaN,NaN,NaN,311,5.0,Hong Kong (School Demo),NaN,NaN,NaN,7d353579-07af-5479-bd50-14333cf4eb76,VTC,vtchk


In [3]:
def remove_extra_spaces(text):
  """Removes extra spaces from a string, including leading, trailing, and multiple spaces between words.

  Args:
      text: The string to process.

  Returns:
      A new string with extra spaces removed.
  """
  return " ".join(text.split()).strip()

def is_none_or_nan(value):
  return value is None or pd.isnull(value) or value == 'nan' 

def is_none_or_nan_zero(value):
  return value is None or pd.isnull(value) or len(value) == 0 or remove_extra_spaces(str(value)) == '' or value == 'nan'

In [4]:
sys_df = df
# site
for i in range(len(sys_df)):
    if np.isnan(sys_df.loc[i, 'parent_system_id']) \
        and is_none_or_nan(sys_df.loc[i, 'component_of_id']):         
            print(sys_df.loc[i, "name_x"])

VTC


In [8]:
# pov
# folder system which has no meterid, defined as hierachy.node_pov
for i in range(len(sys_df)):
    if not is_none_or_nan(sys_df.loc[i, 'parent_system_id']) \
        and is_none_or_nan_zero(str(sys_df.loc[i, 'composition_expression'])) \
        and is_none_or_nan_zero(str(sys_df.loc[i, 'meter_id'])):
            print(f"{sys_df.loc[i, 'id_x']} {sys_df.loc[i, 'name_x']}")

16148 IAQ Sensor 1101
16214 19/F
16149 IAQ Sensor 1107
16216 IAQ Sensor- Johnathan Lee's Office
16119 11/F
16120 1101
16124 1107


In [6]:
for i in range(len(sys_df)):
    if not is_none_or_nan(sys_df.loc[i, 'parent_system_id']):
        ot = f"[POV] id [{sys_df.loc[i, 'id_x']}]"
        ce = str(sys_df.loc[i, 'composition_expression'])
        print(f"{ot} ce:{ce}, check-ce:{is_none_or_nan_zero(ce)}")
        mi = str(sys_df.loc[i, 'meter_id'])
        print(f"{ot} mi:{mi}, check-mi:{is_none_or_nan_zero(mi)}")

[POV] id [16127] ce:nan, check-ce:True
[POV] id [16127] mi:625.0, check-mi:False
[POV] id [16148] ce:nan, check-ce:True
[POV] id [16148] mi:nan, check-mi:True
[POV] id [16214] ce:nan, check-ce:True
[POV] id [16214] mi:nan, check-mi:True
[POV] id [16149] ce:nan, check-ce:True
[POV] id [16149] mi:nan, check-mi:True
[POV] id [16216] ce:nan, check-ce:True
[POV] id [16216] mi:nan, check-mi:True
[POV] id [16119] ce:nan, check-ce:True
[POV] id [16119] mi:nan, check-mi:True
[POV] id [16120] ce:nan, check-ce:True
[POV] id [16120] mi:nan, check-mi:True
[POV] id [16121] ce:nan, check-ce:True
[POV] id [16121] mi:626.0, check-mi:False
[POV] id [16122] ce:nan, check-ce:True
[POV] id [16122] mi:626.0, check-mi:False
[POV] id [16123] ce:nan, check-ce:True
[POV] id [16123] mi:626.0, check-mi:False
[POV] id [16125] ce:nan, check-ce:True
[POV] id [16125] mi:625.0, check-mi:False
[POV] id [16126] ce:nan, check-ce:True
[POV] id [16126] mi:625.0, check-mi:False
[POV] id [16124] ce:nan, check-ce:True
[POV] i

In [ ]:
code = readOption("code")
components_binding = readOption("components_binding")        
host=readOption("databases.organization.host")
port=readOption("databases.organization.port")
database=readOption("databases.organization.database")
user=readOption("databases.organization.username")
password=readOption("databases.organization.password")
org = Organization(host=host, port=port, user=user, password=password, database=database)

tenant = org.Tenant(code)

print(f"{tenant.name.values[0]} | {tenant.company_code.values[0]} | {tenant.id.values[0]}")

In [ ]:
# generate tenant tree in dataflow
dataFlow = DataFlow(code, components_binding, simulation=True)
df = dataFlow.PreparingData()
# df = dataFlow.LoadData()
logger.info("====================== create_nodes_and_datapoints ==========================")
dataFlow.create_nodes_and_datapoints(df, tenant.id.values[0])

In [ ]:
ssd = dataFlow.hr.simulation_sys_df
print(ssd.shape)
with pd.option_context('display.max_columns', None):
    display(ssd[:2])

In [ ]:
srd = dataFlow.hr.simulation_relations_df
print(srd.shape)
with pd.option_context('display.max_columns', None):
    display(srd[:10])

In [ ]:
site_node_id = srd[srd['parent_type']=='SITE']['parent_id'].iloc[0]
print(site_node_id)

In [ ]:
hr = dataFlow.hr
logger.debug("====================== Simulate TenantTree XML ==========================")    
tenantTree = dataFlow.hr.TenantSimulateTree(
                        # tenant.id.values[0],
                        site_node_id,
                        tenant.name.values[0],
                        tenant.company_code.values[0])

site_path = f"./output/{code}"
hr.SaveToXml(tenantTree, f"{site_path}/sim_new_{code}.xml") 

In [ ]:
host=readOption("databases.ems.host")
port=readOption("databases.ems.port")
database=readOption("databases.ems.database")
user=readOption("databases.ems.username")
password=readOption("databases.ems.password")    

logger.debug("======================= EMS system tree XML =========================")
ems = EMS(host=host, port=port, user=user, password=password, database=database)
company = ems.company(code)

# logger.debug(f"{company.name.values[0]} | {company.code.values[0]} | {company.id.values[0]}")
companyTree = ems.SystemTree(company.id.values[0],
                                company.name.values[0],
                                company.code.values[0])

ems.SaveToXml(companyTree, f"{site_path}/sim_{code}_system.xml")

In [ ]:
ems = EMS("localhost", "5432", "mangopie", "mangopie", "mangopie")

In [ ]:
company_df = ems.company(code="cdnis")
print(company_df)

In [ ]:
org = Organization()
tenant_df = org.Tenant(code="cdnis")

print(tenant_df)

In [ ]:
joined_tenant = pd.merge(company_df, tenant_df, how="left", left_on="code", right_on="company_code")
tenant_df = joined_tenant[['id_x', 'id_y', 'name_x', 'code']]

new_names = {'id_x': 'company_id', 'id_y': 'tenant_id', 'name_x': 'company', "code": 'company_code'}
tenant_df = tenant_df.rename(columns=new_names)

with pd.option_context('display.max_columns', None):
    display(tenant_df[:3])   

In [ ]:
sys_df = ems.systems(code="cdnis")
# Set options temporarily within the context of `display()`
with pd.option_context('display.max_columns', None):
    display(sys_df[:8])

In [ ]:
# sys_df = ems.virtual_systems(code="cdnis")
# # Set options temporarily within the context of `display()`
# with pd.option_context('display.max_columns', None):
#     display(sys_df[:8])

In [ ]:
energy = Energy("localhost", "5432", "energy", "energy", "energy")

In [ ]:
dp = energy.dataPoint(columns = ["id", "ref_id", "name"])
dp[:3]

In [ ]:
joined_df = pd.merge(sys_df, dp, how="left", left_on="id", right_on="ref_id")


# 查看连接后的DataFrame
with pd.option_context('display.max_columns', None):
    display(joined_df[:8])


In [ ]:
joined_df.loc[joined_df['id_x'] == int(386)]#[['id_y', 'ref_id']]

In [ ]:
joined_df.loc[joined_df['source_key'] == 'ML']

In [ ]:
with pd.option_context('display.max_columns', None):
    display(tenant_df[:3])   

In [ ]:
joined_df = pd.merge(joined_df, tenant_df, how="left", left_on="company_id", right_on="company_id")
# 查看连接后的DataFrame
with pd.option_context('display.max_columns', None):
    display(joined_df[:8])

In [ ]:
def replace_id(df):
  """
  循环替换 DataFrame 中的 {id_xxx}

  Args:
    df: 输入 DataFrame

  Returns:
    替换后的 DataFrame
  """
  import re
  for i in range(df.shape[0]):
    row = df.iloc[i]
    composition_expression = row['composition_expression']
    for match in re.finditer(r'{id_(\d+)}', composition_expression):
      id_xxx = match.group(1)
      use_system_id = df.loc[df['id'] == int(id_xxx), 'use_system_id'].values[0]
      composition_expression = composition_expression.replace(match.group(0), "{id_"+str(use_system_id)+"}")
    df.loc[i, 'composition_expression'] = composition_expression

  return df

# 使用示例
df = pd.DataFrame({
  'id': [13, 63, 8],
  'composition_expression': [
    '1 if  {id_13}  >= 3 else 1 if  {id_63}  >= 3 else {id_8}',
    '1 if ({id_63}*2)+({id_63}*{id_8})  >= 3 else {id_8}',
    '{id_8}'
  ],
  'use_system_id': [100, 200, 300]
})

print(df)
print("----------------------------------------------------------------")
df = replace_id(df)

print(df)


In [ ]:
vdp = energy.virtualDataPoint()
vdp[:3]

In [ ]:
sys_df

In [ ]:
hr = Hierarchy()

In [ ]:
n_dp = hr.nodeDataPoint()

with pd.option_context('display.max_columns', None):
    display(n_dp[:2])

In [ ]:
n_pov = hr.nodePov()
with pd.option_context('display.max_columns', None):
    display(n_pov[:2])

In [ ]:
n_site = hr.nodeSite()
with pd.option_context('display.max_columns', None):
    display(n_site[:2])

In [ ]:
n_relations = hr.nodeRelations()
with pd.option_context('display.max_columns', None):
    display(n_relations[:2])

In [ ]:
# tenantTree = hr.TenantTree("c93789d6-d78e-5382-8817-c3917ca879be")
# tenantTree